<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_9_LangGraph_%E1%84%86%E1%85%A1%E1%86%BA%E1%84%87%E1%85%A9%E1%84%80%E1%85%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph

### LangGraph란 무엇인가요?

LangGraph는 LLM 애플리케이션에 **"순환(Loop)"**과 "제어(Control)" 기능을 부여하는 라이브러리입니다.

- 기존 LangChain: A → B → C (일방통행, 되돌아가기 어려움)

- LangGraph: A ↔ B (필요하면 다시 돌아감, 상태를 기억함)


이번 실습에서는 LLM이 계산이 필요하면 계산기 도구를 쓰고, 다시 돌아와서 답변하는 '순환 구조'를 만들어 봅니다.

In [ ]:
# 1. 라이브러리 설치
!pip install -qU langgraph langchain-openai langchain-community grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
import getpass

# 2. OpenAI API Key 설정
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key 입력: ")

OpenAI API Key 입력: ··········


### 상태(State)와 도구(Tool) 정의
LangGraph의 핵심은 **State(상태)**입니다. 이 상태 객체가 노드들 사이를 흘러다니며 대화 내용(messages)을 계속 업데이트합니다.

add_messages: 새로운 대화가 오면 기존 리스트를 덮어쓰지 않고 **추가(append)**하라는 설정입니다.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# 1. 그래프의 상태(State) 정의
# 메시지 리스트에 새로운 메시지를 계속 추가(append)하는 방식입니다.
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. 사용할 도구(Tool) 정의
# LLM은 수학 계산에 약하므로, 곱셈을 해주는 도구를 쥐어줍니다.
@tool
def multiply(a: int, b: int) -> int:
    """두 정수의 곱셈 결과를 반환합니다. 계산이 필요할 때 사용하세요."""
    return a * b

# 3. LLM과 도구 연결 (Bind)
# gpt-4o-mini 모델이 이 도구의 존재를 알게 됩니다.
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [multiply]
llm_with_tools = llm.bind_tools(tools)

print("상태 정의 및 도구 연결 완료!")

상태 정의 및 도구 연결 완료!


### 노드(Node)와 그래프(Graph) 구성

이제 작업자(Node)를 배치하고 작업 순서(Edge)를 연결합니다.

- Chatbot Node: LLM이 생각하고 답변하는 곳
- Tools Node: LLM이 도구 사용을 요청하면 실제로 실행하는 곳
- Conditional Edge (분기): LLM의 응답을 보고 "도구 쓸래?" 아니면 "답변 하고 끝낼래?"를 결정
- Loop (순환): 도구를 썼으면 반드시 다시 챗봇에게 돌아가서 결과를 보고하도록 연결 (tools -> chatbot)

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

# 1. 챗봇 노드 함수 정의
def chatbot(state: State):
    # 현재까지의 대화 기록(state["messages"])을 보고 다음 말을 생성합니다.
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 2. 그래프 빌더 생성
graph_builder = StateGraph(State)

# 3. 노드 추가 (작업자 배치)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools)) # LangGraph가 제공하는 도구 실행 전용 노드

# 4. 엣지 연결 (작업 순서 연결)
# [시작] -> [챗봇]
graph_builder.set_entry_point("chatbot")

# [챗봇] -> (조건부 분기) -> [도구] 또는 [종료]
# tools_condition: LLM이 도구를 찾으면 'tools'로, 아니면 '__end__'로 보냅니다.
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# [도구] -> [챗봇] (★핵심: 도구 사용 후 다시 챗봇으로 돌아옵니다 = 순환)
graph_builder.add_edge("tools", "chatbot")

# 5. 그래프 컴파일 (실행 가능한 앱으로 변환)
graph = graph_builder.compile()

# 그래프 구조 시각화 (텍스트 형태)
print("그래프 생성 완료! 구조 확인:")
graph.get_graph().print_ascii()

그래프 생성 완료! 구조 확인:
        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +---------+         
          | chatbot |         
          +---------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


### 실전 테스트
이제 만든 챗봇을 테스트해 봅니다.

- Case 1: 도구가 필요 없는 일상 대화
- Case 2: 도구가 필요한 계산 요청 (순환 발생)

실행 결과를 보면 Case 2에서 Chatbot -> Tools -> Chatbot 순서로 이동하는 것을 볼 수 있습니다.

In [ ]:
from langchain_core.messages import HumanMessage

def run_chat(question):
    print(f"\n 질문: {question}")
    print("-" * 40)

    # graph.stream()을 통해 각 단계(Node)가 실행될 때마다 결과를 봅니다.
    for event in graph.stream({"messages": [HumanMessage(content=question)]}):
        for key, value in event.items():
            print(f" [현재 위치: {key}]")

            # 챗봇이 응답한 경우
            if key == "chatbot":
                msg = value['messages'][-1]
                if msg.tool_calls:
                    print(f"   도구 호출 감지! (이름: {msg.tool_calls[0]['name']}, 값: {msg.tool_calls[0]['args']})")
                else:
                    print(f"   최종 답변: {msg.content}")

            # 도구가 실행된 경우
            elif key == "tools":
                msg = value['messages'][-1]
                print(f"   도구 실행 결과: {msg.content}")

In [ ]:
# --- Case 1 테스트 실행 ---
run_chat("안녕? 너는 어떤 도구를 쓸 수 있어?")


 질문: 안녕? 너는 어떤 도구를 쓸 수 있어?
----------------------------------------
 [현재 위치: chatbot]
   최종 답변: 안녕하세요! 저는 두 정수의 곱셈 결과를 계산할 수 있는 도구를 사용할 수 있습니다. 필요한 계산이 있다면 말씀해 주세요!


In [ ]:
# --- Case 2 테스트 실행 ---
run_chat("123 곱하기 456은 뭐야?")


 질문: 123 곱하기 456은 뭐야?
----------------------------------------
 [현재 위치: chatbot]
   도구 호출 감지! (이름: multiply, 값: {'a': 123, 'b': 456})
 [현재 위치: tools]
   도구 실행 결과: 56088
 [현재 위치: chatbot]
   최종 답변: 123 곱하기 456은 56,088입니다.
